In [3]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.callbacks import EarlyStopping
import codecs

import io
import random as rn
import tensorflow as tf
from gensim.models.keyedvectors import KeyedVectors

/root/anaconda3/envs/ser36/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [5]:

# text feature and labels
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
# x_train_text = np.load(path+'x_train_text.npy')
# vad = np.load(path+'y_egemaps.npy')
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')

data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)
# standardization
scaled_feature = False

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
EMBEDDING_DIM = 300
word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

file_loc = '../../dimensionalSM/data/GoogleNews-vectors-negative300.bin'
print (file_loc)

word_vectors = KeyedVectors.load_word2vec_format(file_loc, binary=True)

g_word_embedding_matrix = {}
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=nb_words:
        continue
    try:
        gembedding_vector = word_vectors[word]
        g_word_embedding_matrix[i] = gembedding_vector
    except KeyError:
        g_word_embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))
del(word_vectors)



scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

Found 3437 unique tokens
../../dimensionalSM/data/GoogleNews-vectors-negative300.bin
G Null word embeddings: 1


In [6]:
# split train/test
split = 7869

In [7]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [8]:
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    #net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    #out_text = Dropout(0.5)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
model = text_model1()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     1031400     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 256)     439296      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 128)          164352      bidirectional_1[0][0]            
___________

In [9]:
# main function for for LOSO
def main(time):
    model = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_text[:split], 
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_text[split:], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    weight = []
    for time in range(30):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('text_blstm_W2V_7869_no_dropout.csv', header=['time','v','a','d', 'ave'])



Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 54s 9ms/step - loss: 1.8546 - v_loss: 0.5983 - a_loss: 0.6306 - d_loss: 0.6263 - v_ccc: 0.4019 - a_ccc: 0.3695 - d_ccc: 0.3739 - val_loss: 2.0168 - val_v_loss: 0.7062 - val_a_loss: 0.6600 - val_d_loss: 0.6679 - val_v_ccc: 0.2955 - val_a_ccc: 0.3474 - val_d_ccc: 0.3403
Epoch 2/50
6295/6295 [==============================] - 51s 8ms/step - loss: 1.4109 - v_loss: 0.3956 - a_loss: 0.5121 - d_loss: 0.5038 - v_ccc: 0.6047 - a_ccc: 0.4880 - d_ccc: 0.4965 - val_loss: 1.9462 - val_v_loss: 0.6615 - val_a_loss: 0.6466 - val_d_loss: 0.6584 - val_v_ccc: 0.3415 - val_a_ccc: 0.3618 - val_d_ccc: 0.3506
Epoch 3/50
6295/6295 [==============================] - 51s 8ms/step - loss: 1.2650 - v_loss: 0.3399 - a_loss: 0.4668 - d_loss: 0.4580 - v_ccc: 0.6601 - a_ccc: 0.5330 - d_ccc: 0.5418 - val_loss: 1.9850 - val_v_loss: 0.6179 - val_a_loss: 0.6707 - val_d_loss: 0.7193 - val_v_ccc: 0.3892 - val_a_ccc: 0.3

Epoch 9/50
6295/6295 [==============================] - 57s 9ms/step - loss: 0.7638 - v_loss: 0.1851 - a_loss: 0.2862 - d_loss: 0.2930 - v_ccc: 0.8151 - a_ccc: 0.7140 - d_ccc: 0.7072 - val_loss: 1.8492 - val_v_loss: 0.5561 - val_a_loss: 0.6597 - val_d_loss: 0.6615 - val_v_ccc: 0.4517 - val_a_ccc: 0.3503 - val_d_ccc: 0.3487
Epoch 10/50
6295/6295 [==============================] - 57s 9ms/step - loss: 0.7106 - v_loss: 0.1731 - a_loss: 0.2669 - d_loss: 0.2704 - v_ccc: 0.8268 - a_ccc: 0.7331 - d_ccc: 0.7296 - val_loss: 1.8667 - val_v_loss: 0.5594 - val_a_loss: 0.6592 - val_d_loss: 0.6738 - val_v_ccc: 0.4488 - val_a_ccc: 0.3479 - val_d_ccc: 0.3367
Epoch 11/50
6295/6295 [==============================] - 56s 9ms/step - loss: 0.6565 - v_loss: 0.1600 - a_loss: 0.2506 - d_loss: 0.2465 - v_ccc: 0.8399 - a_ccc: 0.7499 - d_ccc: 0.7536 - val_loss: 1.8486 - val_v_loss: 0.5440 - val_a_loss: 0.6565 - val_d_loss: 0.6734 - val_v_ccc: 0.4642 - val_a_ccc: 0.3500 - val_d_ccc: 0.3372
Epoch 12/50
6295/6295 [

6295/6295 [==============================] - 60s 9ms/step - loss: 1.8387 - v_loss: 0.5807 - a_loss: 0.6277 - d_loss: 0.6293 - v_ccc: 0.4190 - a_ccc: 0.3719 - d_ccc: 0.3703 - val_loss: 1.9665 - val_v_loss: 0.6900 - val_a_loss: 0.6550 - val_d_loss: 0.6458 - val_v_ccc: 0.3171 - val_a_ccc: 0.3532 - val_d_ccc: 0.3632
Epoch 2/50
6295/6295 [==============================] - 55s 9ms/step - loss: 1.4166 - v_loss: 0.3934 - a_loss: 0.5155 - d_loss: 0.5077 - v_ccc: 0.6066 - a_ccc: 0.4847 - d_ccc: 0.4922 - val_loss: 1.9200 - val_v_loss: 0.6430 - val_a_loss: 0.6576 - val_d_loss: 0.6435 - val_v_ccc: 0.3622 - val_a_ccc: 0.3513 - val_d_ccc: 0.3665
Epoch 3/50
6295/6295 [==============================] - 54s 9ms/step - loss: 1.2607 - v_loss: 0.3326 - a_loss: 0.4687 - d_loss: 0.4593 - v_ccc: 0.6675 - a_ccc: 0.5312 - d_ccc: 0.5406 - val_loss: 1.8749 - val_v_loss: 0.6043 - val_a_loss: 0.6313 - val_d_loss: 0.6602 - val_v_ccc: 0.3991 - val_a_ccc: 0.3769 - val_d_ccc: 0.3490
Epoch 4/50
6295/6295 [==============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 55s 9ms/step - loss: 0.4947 - v_loss: 0.1194 - a_loss: 0.1906 - d_loss: 0.1847 - v_ccc: 0.8804 - a_ccc: 0.8095 - d_ccc: 0.8153 - val_loss: 1.8585 - val_v_loss: 0.5553 - val_a_loss: 0.6604 - val_d_loss: 0.6669 - val_v_ccc: 0.4520 - val_a_ccc: 0.3468 - val_d_ccc: 0.3428
Epoch 16/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.4709 - v_loss: 0.1130 - a_loss: 0.1819 - d_loss: 0.1759 - v_ccc: 0.8870 - a_ccc: 0.8181 - d_ccc: 0.8241 - val_loss: 1.8947 - val_v_loss: 0.5571 - val_a_loss: 0.6724 - val_d_loss: 0.6878 - val_v_ccc: 0.4510 - val_a_ccc: 0.3327 - val_d_ccc: 0.3215
Epoch 17/50
2170/2170 [==============================] - 5s 2ms/step
[0.36934104561805725, 0.37592315673828125, 0.39699193835258484]
4
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 59s 9ms/step - loss: 1.8434 - v_loss: 0.5926 - a_loss: 0.6226 - d_loss: 0.6277 - v_ccc: 0.4072 - a_ccc: 0.3770 - d_ccc: 0

...

6295/6295 [==============================] - 41s 7ms/step - loss: 1.2607 - v_loss: 0.3313 - a_loss: 0.4671 - d_loss: 0.4617 - v_ccc: 0.6685 - a_ccc: 0.5329 - d_ccc: 0.5379 - val_loss: 1.9923 - val_v_loss: 0.6583 - val_a_loss: 0.6554 - val_d_loss: 0.7001 - val_v_ccc: 0.3473 - val_a_ccc: 0.3528 - val_d_ccc: 0.3076
Epoch 4/50
6295/6295 [==============================] - 41s 7ms/step - loss: 1.1459 - v_loss: 0.2951 - a_loss: 0.4277 - d_loss: 0.4225 - v_ccc: 0.7048 - a_ccc: 0.5722 - d_ccc: 0.5772 - val_loss: 1.8468 - val_v_loss: 0.5856 - val_a_loss: 0.6289 - val_d_loss: 0.6539 - val_v_ccc: 0.4184 - val_a_ccc: 0.3796 - val_d_ccc: 0.3551
Epoch 5/50
6295/6295 [==============================] - 41s 7ms/step - loss: 1.0470 - v_loss: 0.2605 - a_loss: 0.3924 - d_loss: 0.3940 - v_ccc: 0.7395 - a_ccc: 0.6075 - d_ccc: 0.6059 - val_loss: 1.8985 - val_v_loss: 0.6278 - val_a_loss: 0.6326 - val_d_loss: 0.6618 - val_v_ccc: 0.3786 - val_a_ccc: 0.3755 - val_d_ccc: 0.3474
Epoch 6/50
6295/6295 [==============

Epoch 8/50
6295/6295 [==============================] - 45s 7ms/step - loss: 0.8379 - v_loss: 0.2056 - a_loss: 0.3144 - d_loss: 0.3178 - v_ccc: 0.7945 - a_ccc: 0.6856 - d_ccc: 0.6821 - val_loss: 1.8596 - val_v_loss: 0.5570 - val_a_loss: 0.6383 - val_d_loss: 0.6830 - val_v_ccc: 0.4440 - val_a_ccc: 0.3690 - val_d_ccc: 0.3273
Epoch 9/50
6295/6295 [==============================] - 44s 7ms/step - loss: 0.7796 - v_loss: 0.1906 - a_loss: 0.2963 - d_loss: 0.2926 - v_ccc: 0.8094 - a_ccc: 0.7037 - d_ccc: 0.7073 - val_loss: 1.8349 - val_v_loss: 0.5548 - val_a_loss: 0.6351 - val_d_loss: 0.6659 - val_v_ccc: 0.4474 - val_a_ccc: 0.3728 - val_d_ccc: 0.3448
Epoch 10/50
6295/6295 [==============================] - 45s 7ms/step - loss: 0.7259 - v_loss: 0.1817 - a_loss: 0.2740 - d_loss: 0.2705 - v_ccc: 0.8186 - a_ccc: 0.7260 - d_ccc: 0.7294 - val_loss: 1.8711 - val_v_loss: 0.5490 - val_a_loss: 0.6544 - val_d_loss: 0.6881 - val_v_ccc: 0.4554 - val_a_ccc: 0.3526 - val_d_ccc: 0.3209
Epoch 11/50
6295/6295 [=

6295/6295 [==============================] - 54s 9ms/step - loss: 1.8400 - v_loss: 0.5872 - a_loss: 0.6226 - d_loss: 0.6297 - v_ccc: 0.4123 - a_ccc: 0.3772 - d_ccc: 0.3705 - val_loss: 1.9929 - val_v_loss: 0.7064 - val_a_loss: 0.6644 - val_d_loss: 0.6427 - val_v_ccc: 0.2976 - val_a_ccc: 0.3437 - val_d_ccc: 0.3658
Epoch 2/50
6295/6295 [==============================] - 45s 7ms/step - loss: 1.4316 - v_loss: 0.3924 - a_loss: 0.5238 - d_loss: 0.5155 - v_ccc: 0.6075 - a_ccc: 0.4764 - d_ccc: 0.4845 - val_loss: 1.9113 - val_v_loss: 0.6357 - val_a_loss: 0.6436 - val_d_loss: 0.6535 - val_v_ccc: 0.3680 - val_a_ccc: 0.3652 - val_d_ccc: 0.3555
Epoch 3/50
6295/6295 [==============================] - 45s 7ms/step - loss: 1.2589 - v_loss: 0.3305 - a_loss: 0.4703 - d_loss: 0.4578 - v_ccc: 0.6696 - a_ccc: 0.5296 - d_ccc: 0.5419 - val_loss: 1.9060 - val_v_loss: 0.5987 - val_a_loss: 0.6360 - val_d_loss: 0.6912 - val_v_ccc: 0.4051 - val_a_ccc: 0.3719 - val_d_ccc: 0.3169
Epoch 4/50
6295/6295 [==============

Epoch 7/50
6295/6295 [==============================] - 46s 7ms/step - loss: 0.8973 - v_loss: 0.2229 - a_loss: 0.3342 - d_loss: 0.3403 - v_ccc: 0.7771 - a_ccc: 0.6658 - d_ccc: 0.6598 - val_loss: 1.8848 - val_v_loss: 0.5912 - val_a_loss: 0.6441 - val_d_loss: 0.6732 - val_v_ccc: 0.4152 - val_a_ccc: 0.3632 - val_d_ccc: 0.3367
Epoch 8/50
6295/6295 [==============================] - 46s 7ms/step - loss: 0.8281 - v_loss: 0.2003 - a_loss: 0.3118 - d_loss: 0.3160 - v_ccc: 0.7996 - a_ccc: 0.6881 - d_ccc: 0.6842 - val_loss: 1.8188 - val_v_loss: 0.5690 - val_a_loss: 0.6229 - val_d_loss: 0.6526 - val_v_ccc: 0.4382 - val_a_ccc: 0.3850 - val_d_ccc: 0.3581
Epoch 9/50
6295/6295 [==============================] - 45s 7ms/step - loss: 0.7606 - v_loss: 0.1849 - a_loss: 0.2834 - d_loss: 0.2923 - v_ccc: 0.8152 - a_ccc: 0.7166 - d_ccc: 0.7076 - val_loss: 1.9044 - val_v_loss: 0.5787 - val_a_loss: 0.6863 - val_d_loss: 0.6634 - val_v_ccc: 0.4297 - val_a_ccc: 0.3189 - val_d_ccc: 0.3471
Epoch 10/50
6295/6295 [==

Epoch 14/50
2170/2170 [==============================] - 4s 2ms/step
[0.373810738325119, 0.4000573456287384, 0.41696786880493164]
29
